## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula

## Read-in

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

## Clean Data

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [6]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [8]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon
1,Ritz Carlton\r4701 N Meridian Ave 601\rMiami B...,5/30/2023,233,"$7,400,000.00",4806,"$1,782.70",Dina Goldentayer,Douglas Elliman,Darin Tansey,Douglas Elliman,Ritz Carlton 4701 N Meridian Ave 601 Miami Beach,Ritz Carlton,4701 N Meridian Ave 601 Miami Beach,"(25.8230161, -80.1312777)",25.823016,-80.131278
2,57 Ocean Condo\r5775 Collins Ave 302\rMiami Beach,5/31/2023,41,"$5,550,000.00",1728,"$3,211.81",Christopher Jude,Fortune Christie's Int'l Real Estate,Christopher Jude,Fortune Christie's Int'l Real Estate,57 Ocean Condo 5775 Collins Ave 302 Miami Beach,,57 Ocean Condo 5775 Collins Ave 302 Miami Beach,"(25.8392568, -80.12039469999999)",25.839257,-80.120395
3,Chateau Beach\r17475 Collins Ave 1402\rSunny I...,6/2/2023,19,"$5,300,000.00",3340,"$1,586.83",Cristiano Piquet,Piquet Realty,Jimena Duran,Gimeza A Real Estate Corp.,Chateau Beach 17475 Collins Ave 1402 Sunny Isl...,Chateau Beach,17475 Collins Ave 1402 Sunny Isles Beach,"(25.9378917, -80.12093329999999)",25.937892,-80.120933
4,Grand Bay\r445 Grand Bay Dr 1209\rKey Biscayne,5/30/2023,305,"$5,135,000.00",3480,"$1,475.57",Brigitte Nachtigall,Great Properties International LLC,Amanda Patino,BHHS EWM Realty,Grand Bay 445 Grand Bay Dr 1209 Key Biscayne,Grand Bay,445 Grand Bay Dr 1209 Key Biscayne,"(25.6927596, -80.15979569999999)",25.692760,-80.159796
5,The Murano Portofino\r1000 S Pointe Dr 2202\rM...,5/31/2023,181,"$4,570,000.00",2618,"$1,745.61",Laura Pantano,Interealty Exchange Inc,Colin Feuling,Brown Harris Stevens,The Murano Portofino 1000 S Pointe Dr 2202 Mia...,The Murano Portofino,1000 S Pointe Dr 2202 Miami Beach,"(25.7685326, -80.13755619999999)",25.768533,-80.137556
6,Residence by Armani Casa\r18975 Collins Ave 26...,5/31/2023,84,"$3,900,000.00",4019,"$1,278.69",Leah Barney,Official,Daniel Tzinker,The Agency Florida LLC,Residence by Armani Casa 18975 Collins Ave 260...,Residence by Armani Casa,18975 Collins Ave 2601 Sunny Isles Beach,"(25.9519943, -80.11997)",25.951994,-80.119970
7,Residence by Armani Casa\r18975 Collins Ave 14...,6/2/2023,247,"$3,800,000.00",3481,"$1,399.12",Oren Alexander,Official,Franco Tommasi,The Corcoran Group,Residence by Armani Casa 18975 Collins Ave 140...,Residence by Armani Casa,18975 Collins Ave 1403 Sunny Isles Beach,"(25.9519943, -80.11997)",25.951994,-80.119970
8,Setai Resort & Residences\r101 20th St 2102\rM...,6/1/2023,10,"$2,950,000.00",1360,"$2,226.42",Julian Cohen,Douglas Elliman,Yuliya Kachko,Oceanfront Group Inc,Setai Resort & Residences 101 20th St 2102 Mia...,Setai Resort & Residences,101 20th St 2102 Miami Beach,"(25.7956082, -80.1281603)",25.795608,-80.128160
9,Turnberry Ocean Colony\r16051 Collins Ave 3301...,5/31/2023,106,"$2,900,000.00",2775,"$1,045.05",Bento Queiroz,"Compass Florida, LLC.",Hector Stracuzzi,Beachfront Realty Inc,Turnberry Ocean Colony 16051 Collins Ave 3301 ...,Turnberry Ocean Colony,16051 Collins Ave 3301 Sunny Isles Beach,"(25.926267, -80.12156499999999)",25.926267,-80.121565
10,0Grand Bay\r445 Grand Bay Dr 1202\rKey Biscayne,6/1/2023,25,"$2,900,000.00",2950,$983.05,Douglas Kinsley,Fortune Christie's Int'l Real Estate,Maria Jarque,Fortune Christie's Int'l Real Estate,0Grand Bay 445 Grand Bay Dr 1202 Key Biscayne,,0Grand Bay 445 Grand Bay Dr 1202 Key Biscayne,"(25.6925451, -80.1585582)",25.692545,-80.158558


In [20]:
df.at[10,'building_name']=('Williams Island')
df.at[10,'address_only']=('2600 Island Blvd')

## Format Data

In [21]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

In [46]:
df['Sale Price']

1     $22,000,000.00
2     $10,965,000.00
3      $5,750,000.00
4      $4,210,000.00
5      $3,600,000.00
6      $2,750,000.00
7      $2,525,000.00
8      $2,300,000.00
9      $2,194,000.00
10     $1,700,000.00
Name: Sale Price, dtype: object

In [49]:
df['int_Sale_Price'] = df['Sale Price'].str.replace('$','',regex=False)

In [50]:
df['int_Sale_Price'] = df['int_Sale_Price'].str.replace(',','',regex=False)

In [51]:
df['int_Sale_Price'] = pd.to_numeric(df['int_Sale_Price'])

In [53]:
df['price_per_sqft'] = df['price_per_sqft'].str.replace('$','',regex=False)
df['price_per_sqft'] = df['price_per_sqft'].str.replace(',','',regex=False)
df['price_per_sqft'] = pd.to_numeric(df['price_per_sqft'])

df['days_on_market'] = pd.to_numeric(df['days_on_market'])

## Color-code top sale

In [54]:
### Insert RANK values ###
df['RANK'] = range(1, len(df) + 1)
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [55]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

In [56]:
def popup_html(row):
    building_name = row['building_name']
    price = row['Sale Price']
    days_on_market = row['days_on_market']
    listing_agent = row['Agent']
    buyers_agent = row['Buyer Agent']
    psf = row['price_per_sqft']
    address = row['address_only']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>{}'''.format(price) + '''<br>
    <strong>Days on Market: </strong>{}'''.format(days_on_market) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(listing_agent) + '''<br>
    <strong>Buyer's Agent: </strong>{}'''.format(buyers_agent) + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(psf) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [57]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('May 21st - May 27th')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [58]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon,int_Sale_Price,RANK,COLOR
1,Oceana Bal Harbour\r10201 Collins Ave 2803\rBa...,5/22/2023,91,"$22,000,000.00",5183,4244.65,Ruthie Assouline,Douglas Elliman,T Kimberly Rodstein,Keller Williams Miami Beach Realty,Oceana Bal Harbour 10201 Collins Ave 2803 Bal ...,Oceana Bal Harbour,10201 Collins Ave 2803 Bal Harbour,"(25.8951514, -80.1230855)",25.895151,-80.123086,22000000.0,1,orange
2,Estates the Acqualina\r17975 Collins Ave 2701\...,5/23/2023,61,"$10,965,000.00",4780,2293.93,Karina Batievsky,Acqualina Realty,Marla Cohen,Acqualina Realty,Estates the Acqualina 17975 Collins Ave 2701 S...,Estates the Acqualina,17975 Collins Ave 2701 Sunny Isles Beach,"(25.9423323, -80.12016559999999)",25.942332,-80.120166,10965000.0,2,blue
3,Jade Signature\r16901 Collins Ave 3401\rSunny ...,5/24/2023,117,"$5,750,000.00",3012,1909.03,Cassio Faccin,FACCIN Group Realty,Alfredo Ferro,Envirotek Realty,Jade Signature 16901 Collins Ave 3401 Sunny Is...,Jade Signature,16901 Collins Ave 3401 Sunny Isles Beach,"(25.9327557, -80.1210054)",25.932756,-80.121005,5750000.0,3,blue
4,Residence by Armani Casa\r18975 Collins Ave 25...,5/22/2023,193,"$4,210,000.00",2682,1569.72,Randal Longo,Keys Realty Redefined LLC,Mariana Niro,Serhant,Residence by Armani Casa 18975 Collins Ave 250...,Residence by Armani Casa,18975 Collins Ave 2502 Sunny Isles Beach,"(25.9519943, -80.11997)",25.951994,-80.119970,4210000.0,4,blue
5,Bayside Village\r2221 Fisher Island Dr 3201\rF...,5/22/2023,275,"$3,600,000.00",2744,1311.95,Svetlana Carroll,One Sotheby's Int'l Realty,Jill Eber,Coldwell Banker Realty,Bayside Village 2221 Fisher Island Dr 3201 Fis...,Bayside Village,2221 Fisher Island Dr 3201 Fisher Island,"(25.7589251, -80.143131)",25.758925,-80.143131,3600000.0,5,blue
6,Muse Condo\r17141 Collins Ave 2502\rSunny Isle...,5/25/2023,49,"$2,750,000.00",2042,1346.72,Natalya Grinberg,"Big International Realty, Inc.",Marcio Sousa,The Keyes Company,Muse Condo 17141 Collins Ave 2502 Sunny Isles ...,Muse Condo,17141 Collins Ave 2502 Sunny Isles Beach,"(25.9349392, -80.12091819999999)",25.934939,-80.120918,2750000.0,6,blue
7,Brickell Flatiron\r1000 Brickell Plaza PH-6007...,5/26/2023,8,"$2,525,000.00",1800,1402.78,Michael Wiesenfeld,The Corcoran Group,Ricardo Di Giorgio,Miami Residential Group,Brickell Flatiron 1000 Brickell Plaza PH-6007 ...,Brickell Flatiron,1000 Brickell Plaza PH-6007 Miami,"(25.7642427, -80.1927022)",25.764243,-80.192702,2525000.0,7,blue
8,Paramount Miami World Center\r851 NE 1st Ave 3...,5/25/2023,91,"$2,300,000.00",3450,747.72,Christian Dreyfuss PA,Brown Harris Stevens,Zarko Draskovic,London Foster Realty,Paramount Miami World Center 851 NE 1st Ave 38...,Paramount Miami World Center,851 NE 1st Ave 3805 Miami,"(25.7824272, -80.1918372)",25.782427,-80.191837,2300000.0,8,blue
9,South Pointe Towers Condo\r400 S Pointe Dr 710...,5/25/2023,118,"$2,194,000.00",1559,1407.31,Jorge Martinez,One Sotheby's Int'l Realty,Marci Declaris,Douglas Elliman,South Pointe Towers Condo 400 S Pointe Dr 710 ...,South Pointe Towers Condo,400 S Pointe Dr 710 Miami Beach,"(25.7672946, -80.1341836)",25.767295,-80.134184,2194000.0,9,blue
10,0Williams Island\r2600 Island Blvd 2501\rAventura,5/24/2023,8,"$1,700,000.00",2580,658.91,Joel Matus,"Williams Island Realty, LLC",Nicole Shein,"Williams Island Realty, LLC",0Williams Island 2600 Island Blvd 2501 Aventura,Williams Island,2600 Island Blvd,"(25.9392382, -80.1337739)",25.939238,-80.133774,1700000.0,10,blue


In [59]:
m.save('index.html')

## Data snagger

In [60]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [61]:
### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

Most Expensive
Oceana Bal Harbour , 10201 Collins Ave 2803 Bal Harbour | Price $22,000,000 | $4,245 psf | Listing agent: Ruthie Assouline with Douglas Elliman | Buyer's agent: T Kimberly Rodstein with Keller Williams Miami Beach Realty | Days on market: 91
Least Expensive
Williams Island, 2600 Island Blvd | Price $1,700,000 | $659 psf | Listing agent: Joel Matus with Williams Island Realty, LLC | Buyer's agent: Nicole Shein with Williams Island Realty, LLC | Days on market: 8
Highest Price Per Square Foot
Oceana Bal Harbour , 10201 Collins Ave 2803 Bal Harbour | Price $22,000,000 | $4,245 psf | Listing agent: Ruthie Assouline with Douglas Elliman | Buyer's agent: T Kimberly Rodstein with Keller Williams Miami Beach Realty | Days on market: 91
Lowest Price Per Square Foot
Williams Island, 2600 Island Blvd | Price $1,700,000 | $659 psf | Listing agent: Joel Matus with Williams Island Realty, LLC | Buyer's agent: Nicole Shein with Williams Island Realty, LLC | Days on market: 8
Most Days 

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [62]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [63]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_05272023
